In [36]:
jsonl_path_diff = "gpt2_defects4j/layer0/feature_importance_diff.jsonl"
jsonl_path_safe = "gpt2_defects4j/layer0/feature_importance_safe.jsonl"
jsonl_path_vuln = "gpt2_defects4j/layer0/feature_importance_vuln.jsonl"


In [37]:
import json
import pandas as pd
import numpy as np


In [38]:

def read_jsonl_file(jsonl_path):
    with open(jsonl_path, "r") as f:
        for line in f:
            yield json.loads(line)


def get_training_indexes(diff_df):
    return np.random.choice(diff_df.index, int(len(diff_df) * 0.8), replace=False)

diff_data = list(read_jsonl_file(jsonl_path_diff))
diff_df = pd.DataFrame(diff_data)

safe_data = list(read_jsonl_file(jsonl_path_safe))
safe_df = pd.DataFrame(safe_data)

vuln_data = list(read_jsonl_file(jsonl_path_vuln))
vuln_df = pd.DataFrame(vuln_data)

vuln_df.drop(columns=["labels", "model", "plot_type"], inplace=True)
vuln_df["vuln"] = 1

safe_df.drop(columns=["labels", "model", "plot_type"], inplace=True)
safe_df["vuln"] = 0

train_indexes = get_training_indexes(diff_df)



safe_df_train = safe_df.loc[train_indexes]
safe_df_test = safe_df.drop(train_indexes)

vuln_df_train = vuln_df.loc[train_indexes]
vuln_df_test = vuln_df.drop(train_indexes)


df_train = pd.concat([safe_df_train, vuln_df_train])
df_test = pd.concat([safe_df_test, vuln_df_test])

df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Dont know if we will use this here
diff_df_train = diff_df.loc[train_indexes]
diff_df_test = diff_df.drop(train_indexes)

# Train based on PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)

pca_result = pca.fit_transform(df_train["values"].values.tolist())
df_train_pca = pd.DataFrame(pca_result, columns=[f"pca_{i}" for i in range(10)])
df_train_pca["vuln"] = df_train["vuln"]

pca_result = pca.transform(df_test["values"].values.tolist())
df_test_pca = pd.DataFrame(pca_result, columns=[f"pca_{i}" for i in range(10)])
df_test_pca["vuln"] = df_test["vuln"]

In [23]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train = df_train_pca.drop(columns=["vuln"])
y_train = df_train_pca["vuln"]

X_test = df_test_pca.drop(columns=["vuln"])
y_test = df_test_pca["vuln"]

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.68      0.71        94
           1       0.70      0.76      0.73        94

    accuracy                           0.72       188
   macro avg       0.72      0.72      0.72       188
weighted avg       0.72      0.72      0.72       188



### PCA on diff

In [33]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)

pca_result = pca.fit_transform(diff_df["values"].values.tolist())
diff_df_pca = pd.DataFrame(pca_result, columns=[f"pca_{i}" for i in range(10)])

In [34]:
pca_result = pca.transform(df_train["values"].values.tolist())
df_train_pca = pd.DataFrame(pca_result, columns=[f"pca_{i}" for i in range(10)])
df_train_pca["vuln"] = df_train["vuln"]

pca_result = pca.transform(df_test["values"].values.tolist())
df_test_pca = pd.DataFrame(pca_result, columns=[f"pca_{i}" for i in range(10)])
df_test_pca["vuln"] = df_test["vuln"]

In [35]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train = df_train_pca.drop(columns=["vuln"])
y_train = df_train_pca["vuln"]

X_test = df_test_pca.drop(columns=["vuln"])
y_test = df_test_pca["vuln"]

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.73      0.66      0.69        94
           1       0.69      0.76      0.72        94

    accuracy                           0.71       188
   macro avg       0.71      0.71      0.71       188
weighted avg       0.71      0.71      0.71       188



# Train based on Diff


In [ ]:
train_df_diff = diff_df.loc[train_indexes]
def get_most_important_features(train_df_diff, n=100):
    return train_df_diff.sum(axis=0).sort_values(ascending=False).index[1 : n + 1]
train_df_diff

,index,model,plot_type,sae_id,cache_component,values,labels
425,425,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[feature_0, feature_1, feature_2, feature_3, f..."
260,260,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0153748486191...","[feature_0, feature_1, feature_2, feature_3, f..."
456,456,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[feature_0, feature_1, feature_2, feature_3, f..."
247,247,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[feature_0, feature_1, feature_2, feature_3, f..."
158,158,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.0, 0.0, 0.0019201040267944336, 0....","[feature_0, feature_1, feature_2, feature_3, f..."
...,...,...,...,...,...,...,...
415,415,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[feature_0, feature_1, feature_2, feature_3, f..."
48,48,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.00983026996254921, 0.0, 0.0, 0.0, 0.08...","[feature_0, feature_1, feature_2, feature_3, f..."
61,61,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.017618656158447266, 0.0, 0.0, 0.0...","[feature_0, feature_1, feature_2, feature_3, f..."
212,212,gpt2-small,sae-feature-importance,blocks.0.hook_resid_pre,hook_sae_acts_post,"[0.0, 0.0, 0.030414938926696777, 0.0, 0.0, 0.0...","[feature_0, feature_1, feature_2, feature_3, f..."


In [ ]:
from tqdm import trange
def get_diff_data(diff_jsonl_path):
    diff_data = list(read_jsonl_file(diff_jsonl_path))
    diff_df = pd.DataFrame(diff_data)

    columns = diff_df.columns.to_list()
    columns.remove("values")

    diff_df.drop(columns=columns, inplace=True)

    for i in trange(len(diff_df["values"][0])):
        diff_df[f"feature_{i}"] = diff_df["values"].apply(lambda x: x[i])

    diff_df.drop(columns=["values"], inplace=True)

    return diff_df


train_df_diff = get_diff_data(jsonl_path_diff)
most_important_features = get_most_important_features(train_df_diff, n=10)
# Convert most_important_features to idxs
most_important_features_idxs = [int(feature.split("_")[1]) for feature in most_important_features]
most_important_features_idxs

100%|██████████| 24576/24576 [00:43<00:00, 569.00it/s]


Index(['feature_1352', 'feature_6111', 'feature_21821', 'feature_18861',
       'feature_24401', 'feature_22351', 'feature_22338', 'feature_7101',
       'feature_9237', 'feature_6002'],
      dtype='object')

In [32]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train = df_train["values"].values.tolist()
X_train = [[x[i] for i in most_important_features_idxs] for x in X_train]
y_train = df_train["vuln"]

X_test = df_test["values"].values.tolist()
X_test = [[x[i] for i in most_important_features_idxs] for x in X_test]
y_test = df_test["vuln"]


clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.76      0.60      0.67        94
           1       0.67      0.81      0.73        94

    accuracy                           0.70       188
   macro avg       0.71      0.70      0.70       188
weighted avg       0.71      0.70      0.70       188



## CLassical ML

In [52]:
OUR_STANDARD_TOKENIZER = "meta-llama/Llama-3.1-8B"
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(OUR_STANDARD_TOKENIZER)

In [53]:
defects4j_path = "artifacts/defects4j.csv"
defects4j_df = pd.read_csv(defects4j_path)
gbug_path = "artifacts/gbug-java.csv"
gbug_df = pd.read_csv(gbug_path)
human_eval_path = "artifacts/humaneval.csv"
human_eval_df = pd.read_csv(human_eval_path)

In [89]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
from tqdm import tqdm
tqdm.pandas()
gbug_df["tokenized_before"] = gbug_df["func_before"].progress_apply(lambda x: tokenizer(x, return_tensors="pt", padding="max_length", truncation=True, max_length=5000
                                                                      )["input_ids"])
gbug_df["tokenized_after"] = gbug_df["func_after"].progress_apply(lambda x: tokenizer(x, return_tensors="pt",padding="max_length",truncation=True, max_length=5000
                                                                      )["input_ids"])
defects4j_df["tokenized_before"] = defects4j_df["func_before"].progress_apply(lambda x: tokenizer(x, return_tensors="pt", padding="max_length",truncation=True, max_length=5000
                                                                        )["input_ids"])
defects4j_df["tokenized_after"] = defects4j_df["func_after"].progress_apply(lambda x: tokenizer(x, return_tensors="pt", padding="max_length",truncation=True, max_length=5000
                                                                        )["input_ids"])
human_eval_df["tokenized_before"] = human_eval_df["func_before"].progress_apply(lambda x: tokenizer(x, return_tensors="pt", padding="max_length",truncation=True, max_length=5000
                                                                        )["input_ids"])
human_eval_df["tokenized_after"] = human_eval_df["func_after"].progress_apply(lambda x: tokenizer(x, return_tensors="pt", padding="max_length", truncation=True, max_length=5000
                                                                        )["input_ids"])



100%|██████████| 162/162 [00:00<00:00, 928.18it/s]


In [90]:
gbug_df["tokenized_before"].iloc[0].shape

torch.Size([1, 5000])

In [91]:
gbug_df_classical = pd.DataFrame()

for row in gbug_df.iterrows():
    row = row[1]
    gbug_df_classical = pd.concat([gbug_df_classical, pd.DataFrame({"tokens": row["tokenized_before"].tolist(), "vuln": 1}, index=[0])])
    gbug_df_classical = pd.concat([gbug_df_classical, pd.DataFrame({"tokens": row["tokenized_after"].tolist(), "vuln": 0}, index=[0])])

defects4j_df_classical = pd.DataFrame()

for row in defects4j_df.iterrows():
    row = row[1]
    defects4j_df_classical = pd.concat([defects4j_df_classical, pd.DataFrame({"tokens": row["tokenized_before"].tolist(), "vuln": 1}, index=[0])])
    defects4j_df_classical = pd.concat([defects4j_df_classical, pd.DataFrame({"tokens": row["tokenized_after"].tolist(), "vuln": 0}, index=[0])])

human_eval_df_classical = pd.DataFrame()

for row in human_eval_df.iterrows():
    row = row[1]
    human_eval_df_classical = pd.concat([human_eval_df_classical, pd.DataFrame({"tokens": row["tokenized_before"].tolist(), "vuln": 1}, index=[0])])
    human_eval_df_classical = pd.concat([human_eval_df_classical, pd.DataFrame({"tokens": row["tokenized_after"].tolist(), "vuln": 0}, index=[0])])

In [92]:
# shuffle
gbug_df_classical = gbug_df_classical.sample(frac=1).reset_index(drop=True)
defects4j_df_classical = defects4j_df_classical.sample(frac=1).reset_index(drop=True)
human_eval_df_classical = human_eval_df_classical.sample(frac=1).reset_index(drop=True)

# split
train_indexes = np.random.choice(gbug_df_classical.index, int(len(gbug_df_classical) * 0.8), replace=False)

gbug_df_train = gbug_df_classical.loc[train_indexes]
gbug_df_test = gbug_df_classical.drop(train_indexes)

defects4j_df_train = defects4j_df_classical.loc[train_indexes]
defects4j_df_test = defects4j_df_classical.drop(train_indexes)

human_eval_df_train = human_eval_df_classical.loc[train_indexes]
human_eval_df_test = human_eval_df_classical.drop(train_indexes)

In [ ]:
# Classical logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import torch

X_train = gbug_df_train["tokens"].values.tolist()
X_train = [torch.tensor(x) for x in X_train]
y_train = gbug_df_train["vuln"]

X_test = gbug_df_test["tokens"].values.tolist()
X_test = [torch.tensor(x) for x in X_test]
y_test = gbug_df_test["vuln"]


# Do grid search
from sklearn.model_selection import GridSearchCV

param_grid = {
    "C": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    "max_iter": [100, 1000, 10000],
    "solver": ["newton-cg", "lbfgs"]
}

clf = GridSearchCV(LogisticRegression(random_state=0), param_grid, cv=5, verbose=3)  # Verbose level 3 for detailed output

# Do verbose to see progress
clf.fit(X_train, y_train)

print(clf.best_params_)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 1/5] END C=1e-05, max_iter=100, solver=newton-cg;, score=0.542 total time=   1.9s
[CV 2/5] END C=1e-05, max_iter=100, solver=newton-cg;, score=0.638 total time=   2.3s
[CV 3/5] END C=1e-05, max_iter=100, solver=newton-cg;, score=0.872 total time=   1.1s
[CV 4/5] END C=1e-05, max_iter=100, solver=newton-cg;, score=0.638 total time=   0.9s
[CV 5/5] END C=1e-05, max_iter=100, solver=newton-cg;, score=0.787 total time=   2.6s
[CV 1/5] END C=1e-05, max_iter=100, solver=lbfgs;, score=0.479 total time=   0.3s
[CV 2/5] END C=1e-05, max_iter=100, solver=lbfgs;, score=0.617 total time=   0.2s
[CV 3/5] END C=1e-05, max_iter=100, solver=lbfgs;, score=0.702 total time=   0.2s
[CV 4/5] END C=1e-05, max_iter=100, solver=lbfgs;, score=0.660 total time=   0.3s
[CV 5/5] END C=1e-05, max_iter=100, solver=lbfgs;, score=0.660 total time=   0.2s
[CV 1/5] END C=1e-05, max_iter=100, solver=liblinear;, score=0.562 total time=   1.0s
[CV 2/5] EN

KeyboardInterrupt: 